In [ ]:
import os
import pandas as pd
import numpy as np

import tensorflow as tf
import tensorflow_addons as tfa

from tensorflow import keras

from tensorflow.keras import layers
from tensorflow.keras import models

from keras.constraints import maxnorm
from keras.models import model_from_json
from keras.models import load_model

from tensorflow.keras.wrappers.scikit_learn import KerasRegressor

from skopt import BayesSearchCV

from sklearn import preprocessing
from sklearn import pipeline

import time
import joblib

from joblib import dump

In [ ]:
X_treinamento = pd.read_json(r'X_treinamento.json')
X_teste = pd.read_json      (r'X_teste.json')
treino_json = pd.read_json  (r'y_treinamento.json', typ='series')
teste_json = pd.read_json   (r'y_teste.json', typ='series')

In [ ]:
a = []
b = []

for i in range(treino_json.size):
    a.append(list(treino_json[i]))
    
for i in range(teste_json.size):
    b.append(list(teste_json[i]))

In [ ]:
y_treinamento = np.array(a, dtype='float32')
y_teste = np.array(b, dtype='float32')

In [ ]:
def build_model(
    optimizer,
    learning_rate,
    activation,
    init_mode,
    dropout_rate,
    weight_constraint,
    neurons,
    regularizers
):
    model = models.Sequential()
    
    # layer 1
    model.add(
        layers.Dense(
            neurons,
            activation = activation,
            kernel_initializer=init_mode,
            kernel_constraint=maxnorm(weight_constraint),
            kernel_regularizer=regularizers,
            input_shape=[len(X_treinamento.keys())]
        )
    )
    model.add(layers.Dropout(dropout_rate))
    
    # layer 2
    model.add(layers.Dense(256))
    
    model.compile(
        loss='mse',
        optimizer=optimizer(learning_rate),
        metrics=['mae', 'mse', keras.metrics.RootMeanSquaredError()]
    )
    
    return model

In [ ]:
optimizers = [
    tf.keras.optimizers.Adam,
    tf.keras.optimizers.SGD,
    tf.keras.optimizers.RMSprop,
    tf.keras.optimizers.Adagrad
]

In [ ]:
activation = [
    tf.keras.activations.gelu,
    tf.keras.activations.swish,
    tfa.activations.mish,
    'relu',
    'sigmoid',
    'softplus',
    'softsign',
    'tanh'
]

In [ ]:
batch_size = [6, 9, 18]
epochs = [400, 500, 600, 700]
learning_rate = [10e-1, 10e-2, 10e-3, 10e-4, 10e-5, 10e-6]
init_mode = ['uniform', 'normal', 'glorot_normal', 'glorot_uniform']
dropout_rate = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5]
weight_constraint = [1, 2, 3, 4, 5]
neurons = [128, 256, 512, 768, 1024]
regularizers = [tf.keras.regularizers.l1(0.0001), tf.keras.regularizers.l2(0.0001), tf.keras.regularizers.l1_l2(0.0001)]

In [ ]:
scalers = [
    preprocessing.StandardScaler(),
    preprocessing.MinMaxScaler(),
    preprocessing.MaxAbsScaler(),
    preprocessing.RobustScaler(quantile_range=(25,75)),
    preprocessing.PowerTransformer(method='yeo-johnson'),
    preprocessing.QuantileTransformer(output_distribution='uniform'),
    preprocessing.QuantileTransformer(output_distribution='normal'),
]

In [ ]:
param_grid = {
    'estimator__batch_size': batch_size,
    'estimator__epochs': epochs,
    'estimator__optimizer': optimizers,
    'estimator__learning_rate': learning_rate,
    'estimator__activation': activation,
    'estimator__init_mode': init_mode,
    'estimator__weight_constraint': weight_constraint,
    'estimator__dropout_rate': dropout_rate,
    'estimator__neurons': neurons,
    'estimator__regularizers': regularizers,
    'scaler': scalers
}

In [ ]:
model = KerasRegressor(build_fn=build_model, verbose=1)

In [ ]:
steps = [('scaler', preprocessing.StandardScaler()), 
         ('estimator', model)]

In [ ]:
pipe = pipeline.Pipeline(steps)

In [ ]:
grid = BayesSearchCV(
    estimator = pipe,
    search_spaces = param_grid,
    n_iter = 50,
    n_jobs = 3,
    cv = 10,
    scoring = 'neg_mean_squared_error',
    refit = True
)

In [ ]:
start = time.time()
grid_result = grid.fit(X_treinamento, y_treinamento)
end = time.time()

In [ ]:
print((end-start)/60)

In [ ]:
grid_result.best_params_

In [ ]:
grid_result.best_index_

In [ ]:
hist = pd.DataFrame(grid_result.best_estimator_['estimator'].model.history.history)
hist.to_json(r'history.json')

In [ ]:
cv_results_ = pd.DataFrame(grid_result.cv_results_)
cv_results_.iloc[:, 0:17].to_json(r'cv_results_.json', orient='columns')

In [ ]:
model = grid_result.best_estimator_['estimator'].model
model.save(r'model.h5')
dump(grid_result.best_estimator_['scaler'], r'scaler.joblib') 

In [ ]:
X_treinamento_100 = pd.concat([X_treinamento, X_teste], axis=0)
y_treinamento_100 = pd.concat([treino_json, teste_json], axis=0)

X_treinamento_100.reset_index(drop=True, inplace=True)
y_treinamento_100.reset_index(drop=True, inplace=True)

a100 = []

for i in range(y_treinamento_100.size):
    a100.append(list(y_treinamento_100[i]))
    
y_treinamento_100 = np.array(a100, dtype='float32')

In [ ]:
grid_result.best_estimator_.fit(X_treinamento_100, y_treinamento_100)

In [ ]:
model100 = grid_result.best_estimator_['estimator'].model
model100.save(r'model-100.h5')
dump(grid_result.best_estimator_['scaler'], r'scaler-100.joblib') 